# Performance analysis and anova

written by Jasmin L. Walter

In [2]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


## import


In [14]:
import copy  # copy big/deep objects by value
import csv
import datetime  # datetime operations
import itertools  # operate with iterators
import json  # read/write from/into json format
import math
import os  # OS operations (read/write files/folders)
import sys
import time
import warnings  # hide warnings
from collections import Counter
from itertools import groupby
import matplotlib

# process parallelization
from multiprocessing import Manager, Pool, RawArray, cpu_count
from os.path import exists

import matplotlib.pyplot as plt  # mother of plots focr Python

# import mlxtend
import numpy as np  # array/matrix operations (e.g. linear algebra)
import pandas as pd  # operate with dataframes
from statsmodels.stats.anova import AnovaRM

import scipy.stats
import seaborn as sns  # matplotlib plotting nice with shortcuts
from IPython.display import Markdown, display  # print nicely
from ipywidgets import IntProgress
#from matplotlib.pyplot import cm
from scipy.signal import savgol_coeffs
from tqdm.notebook import tqdm, trange  # mother of progressbars
from scipy.stats import ks_2samp
from multiprocessing import Pool, cpu_count

import scipy.signal

In [7]:
savepath = "F:/WestbrookProject/Spa_Re/control_group/Analysis/P2B_controls_analysis"


In [8]:
# Participant list of all participants that participated 5 sessions x 30 min
# in Westbrook city
PartList = [1004, 1005, 1008, 1010, 1011, 1013, 1017, 1018, 1019, 1021, 1022, 1023, 1054, 1055, 1056, 1057, 1058, 1068, 1069, 1072, 1073, 1074, 1075, 1077, 1079, 1080]


In [11]:

dataP2B = pd.read_csv(os.path.join(savepath, "overviewTable_P2B_Prep_complete.csv"))


In [12]:
dataP2B.head()

,SubjectID,TimeStampBegin,TrialDuration,StartingPositionIndex,StartBuildingName,TargetBuildingName,RecalculatedAngle,DistancePart2TargetBuilding,NodeDegreeStartBuilding,NodeDegreeTargetBuilding,...,TargetBuildingDwellingTime,Mean_egocentric_global,Mean_survey,Mean_cardinal,TrialTime,RouteID,TrialOrder,TrialSequence,TrialSequence_SameStart14,TrialSequence_SameStart7
0,1004,1.620057e+09,24.381341,7,TaskBuilding_8,TaskBuilding_3,172.355456,221.415804,7,11,...,28.765485,5.8,4.857143,1.5,0.00000,RouteID_52,1,1,1,1
1,1004,1.620057e+09,14.068027,7,TaskBuilding_8,TaskBuilding_2,56.020996,199.514222,7,5,...,66.331504,5.8,4.857143,1.5,24.64670,RouteID_51,1,2,2,2
2,1004,1.620057e+09,12.694109,7,TaskBuilding_8,TaskBuilding_1,10.721127,336.932289,7,13,...,40.847034,5.8,4.857143,1.5,38.98158,RouteID_50,1,3,3,3
3,1004,1.620057e+09,17.271687,7,TaskBuilding_8,TaskBuilding_7,153.416261,196.602196,7,7,...,11.200175,5.8,4.857143,1.5,51.94204,RouteID_56,1,4,4,4
4,1004,1.620057e+09,13.778860,7,TaskBuilding_8,TaskBuilding_5,62.256057,202.430774,7,11,...,33.730450,5.8,4.857143,1.5,69.48157,RouteID_54,1,5,5,5


In [16]:
# Create the data 
dataframe = pd.DataFrame({'Cars': np.repeat([1, 2, 3, 4, 5], 4), 
                          'Oil': np.tile([1, 2, 3, 4], 5), 
                          'Mileage': [36, 38, 30, 29, 
                                      34, 38, 30, 29, 
                                      34, 28, 38, 32, 
                                      38, 34, 20, 44, 
                                      26, 28, 34, 50]}) 
  
# Print the dataframe 
print(dataframe) 

    Cars  Oil  Mileage
0      1    1       36
1      1    2       38
2      1    3       30
3      1    4       29
4      2    1       34
5      2    2       38
6      2    3       30
7      2    4       29
8      3    1       34
9      3    2       28
10     3    3       38
11     3    4       32
12     4    1       38
13     4    2       34
14     4    3       20
15     4    4       44
16     5    1       26
17     5    2       28
18     5    3       34
19     5    4       50


In [17]:
testi = AnovaRM(data=dataframe, depvar='Mileage', 
              subject='Cars', within=['Oil']).fit()

In [18]:
print(testi)

              Anova
    F Value Num DF  Den DF Pr > F
---------------------------------
Oil  0.5679 3.0000 12.0000 0.6466



In [19]:
# Sample data generation
data = {
    'Participant': ['A', 'A', 'B', 'B', 'C', 'C'] * 2,
    'Trial': ['Ti', 'Tb', 'Ti', 'Tb', 'Ti', 'Tb'] * 2,
    'TestOrder': [1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2],
    'Score': [8, 7, 9, 6, 7, 8, 9, 6, 8, 7, 6, 9]
}

df = pd.DataFrame(data)

# Convert necessary columns to categorical
df['Participant'] = df['Participant'].astype('category')
df['Trial'] = df['Trial'].astype('category')
df['TestOrder'] = df['TestOrder'].astype('category')



In [21]:
df.head(20)

,Participant,Trial,TestOrder,Score
0,A,Ti,1,8
1,A,Tb,1,7
2,B,Ti,1,9
3,B,Tb,1,6
4,C,Ti,1,7
5,C,Tb,1,8
6,A,Ti,2,9
7,A,Tb,2,6
8,B,Ti,2,8
9,B,Tb,2,7


In [22]:
# Using AnovaRM for repeated measures
anova_rm = AnovaRM(df, 'Score', 'Participant', within=['Trial', 'TestOrder'])

# Fit the model and print the ANOVA table
anova_results = anova_rm.fit()
print(anova_results)


                   Anova
                F Value Num DF Den DF Pr > F
--------------------------------------------
Trial            0.2500 1.0000 2.0000 0.6667
TestOrder        0.0000 1.0000 2.0000 1.0000
Trial:TestOrder  0.2500 1.0000 2.0000 0.6667



In [23]:
# Fit a Repeated Measures ANOVA model
# Model description: We want to see if there's a change between TestOrder,
# controlled for the fact each Trial needs to be evaluated on its own.
rm = AnovaRM(data=df, depvar='Score', subject='Participant', within=['TestOrder', 'Trial'])
rm_results = rm.fit()
print(rm_results)

# Optionally, if trial differences should be averaged out to focus solely on 'TestOrder'
average_scores = df.groupby(['Participant', 'TestOrder']).agg({'Score': 'mean'}).unstack()
average_diff = average_scores['Score'][2] - average_scores['Score'][1]

print("Mean difference in performance from Measurement 1 to 2:", average_diff.mean())

                   Anova
                F Value Num DF Den DF Pr > F
--------------------------------------------
TestOrder        0.0000 1.0000 2.0000 1.0000
Trial            0.2500 1.0000 2.0000 0.6667
TestOrder:Trial  0.2500 1.0000 2.0000 0.6667

Mean difference in performance from Measurement 1 to 2: 0.0


In [24]:

# Convert necessary columns to categorical
dataP2B['SubjectID'] = dataP2B['SubjectID'].astype('category')
dataP2B['RouteID'] = dataP2B['RouteID'].astype('category')
dataP2B['TrialOrder'] = dataP2B['TrialOrder'].astype('category')



In [25]:
rm = AnovaRM(data=dataP2B, depvar='RecalculatedAngle', subject='SubjectID', within=['TrialOrder', 'RouteID'])
rm_results = rm.fit()
print(rm_results)


                       Anova
                   F Value  Num DF   Den DF  Pr > F
---------------------------------------------------
TrialOrder          6.9671  1.0000   25.0000 0.0141
RouteID             7.1082 55.0000 1375.0000 0.0000
TrialOrder:RouteID  1.1597 55.0000 1375.0000 0.2009



In [26]:

# Optionally, if trial differences should be averaged out to focus solely on 'TestOrder'
average_scores = dataP2B.groupby(['SubjectID', 'TrialOrder']).agg({'RecalculatedAngle': 'mean'}).unstack()
average_diff = average_scores['RecalculatedAngle'][2] - average_scores['RecalculatedAngle'][1]

print("Mean difference in performance from Measurement 1 to 2:", average_diff.mean())

Mean difference in performance from Measurement 1 to 2: -5.148282125968522


In [27]:
average_scores.head()

RecalculatedAngle           
TrialOrder                 1          2
SubjectID                              
1004               66.191136  66.393198
1005               32.812423  38.222044
1008               41.857104  41.805448
1010               62.220685  63.008217
1011               58.219846  47.571798